In [12]:
import math

def setAttrAndCount(examples):
  '''Takes in an array of examples (which are dicts).
  Returns a nested dictionary of the format attr={value1:numOfOccurences, value2: numOfOccurences, ...}'''
  attrDict={}
  for example in examples:
    for attr, value in example.iteritems():
      if attrDict.has_key(attr): # if outerdict has attr as key
        if attrDict[attr].has_key(value): # if the innerdict already has the value as a key
          attrDict[attr][value] += 1
        else: # otherwise create it
          attrDict[attr][value] = 1
      else:
        attrDict[attr] = {value:1} # initialise the dictionary of values
  print attrDict
  return attrDict


def entropy(examples, attribute):
  '''Takes an array of examples and calculates the entropy'''
  allinfo = setAttrAndCount(examples)
  entropy = 0
  for value, count in  allinfo[attribute].iteritems():
    prob = float(count)/len(examples)
    entropy -= prob*math.log(prob, 2)
  return entropy


def infoGain(examples, Attr):
  '''Calculates information gain of an Attribute from the given examples.'''
  allinfo = setAttrAndCount(examples)
  summation = 0
  for value, count in allinfo[Attr].iteritems():
    prob = float(count/len(examples))
    summation += prob*entropy([example for example in examples if example[Attr]==value], "Class")
  gain = entropy(examples, "Class") - summation
  return gain

def chooseAtrr(examples, attributes):
  '''Finds attribute with maximum info gain given a list of examples and attributes.'''
  maxim=0
  maxattr = None # attributes[0] # just a starting pt.
  for x in attributes:
    if infoGain(examples, x) > maxim:
      maxim = infoGain(examples, x)
      maxattr = x
  return maxattr

def mode(examples):
  '''returns majority value of target class'''
  maxcount = 0
  maxval = None
  allinfo = setAttrAndCount(examples) # Can be optimised more. I'm being a little lazy... Don't need to do it for all values, just "Class"
  for value, count in allinfo["Class"].iteritems():
    if count > maxcount:
      maxcount = count
      maxval = value
  return maxval


def without(d, key):
  '''Returns takes in dictionary and key and returns dictionary without given key=value pair.'''
  new_d = d.copy()
  if new_d.has_key(key):
    new_d.pop(key)
  return new_d

def ID3help(root, examples, attrDict, default):
  '''Recursive helper function for main ID3 function.'''
  if not examples:
    root.label = default
    return root
  elif len(set([example["Class"] for example in examples])) == 1:
    root.label = examples[0]["Class"]
    return root
  elif not attrDict:
    root.label = mode(examples)
    return root
  best = chooseAtrr(examples, attrDict.keys())
  print best
  if len(attrDict[best].keys()) == 1:
    root.label = mode(examples)
    return root
  else:
    root.currAttribute = best
    root.label = mode(examples) # doing this for making pruning easy
    for value in attrDict[best]:
      newExamples = [example for example in examples if example[best] == value]
      child = Node()
      child = ID3help(child, newExamples, without(setAttrAndCount(newExamples), best), default)
      # child.prevValue = value
      root.children[value] = child
    return root


def ID3(examples, default):
  '''
  Takes in an array of examples, and returns a tree (an instance of Node)
  trained on the examples.  Each example is a dictionary of attribute:value pairs,
  and the target class variable is a special attribute with the name "Class".
  Any missing attributes are denoted with a value of "?"
  '''
  attrDict = setAttrAndCount(examples)
  root = Node()
  return ID3help(root, examples, without(attrDict, "Class"), default)

# def removeChildren(root, attrToRemove): # redundant, we don't use this anymore
#   '''A DFS Function that searches for the node-to-prune in the tree we found'''
#   stack = []
#   stack.append(root)
#   while len(stack) != 0:
#     currnode = stack.pop()
#     if currnode.currAttribute == attrToRemove:
#       currnode.children = {}
#       break
#     if bool(currnode.children):
#       continue
#     for key in currnode.children:
#       stack.append(currnode.children[key])
#   return root

# def prunehelp(node, examples, acc):
#   '''Helper function for prune that takes in a root node of a tree, a validation set and the accuracy of the previous tree'''
#   accuracy = test(node, examples)
#   nodeToPrune=None
#   if acc < accuracy:
#     print "accuracy base case executed"
#     return node
#   elif not node.children: # return if tree is just single node
#     return node
#   currMax = accuracy
#   stack = [] # stack for DFS.
#   stack.append(node)
#   while len(stack) != 0: # At this point, we're just iterating through nodes to see pruning which one gives us the greatest gain in accuracy
#     currnode = stack.pop()
#     # currAcc = test(removeChildren(node, currnode.currAttribute), examples) # the other inefficient method I was trying
#     temp = currnode.children # temporarily storing children
#     if not bool(temp): # if no children
#       continue
#     currnode.children = {} # pruning it to see what accuracy it gives us
#     currAcc = test(currnode, examples)
#     if currAcc >= currMax:
#       currMax = currAcc
#       nodeToPrune = currnode
#     currnode.children = temp # un-pruning it to test other nodes
#     for key in currnode.children:
#       stack.append(currnode.children[key])
#   if nodeToPrune==None:
#       return node
#   # newTree = removeChildren(node, nodeToPrune.currAttribute)
#   nodeToPrune.children = {}
#   # return prunehelp(newTree, examples, accuracy)
#   return prunehelp(node, examples, accuracy)

def prunehelp(node, examples, acc):
  '''Helper function for prune that takes in a root node of a tree, a validation set and the accuracy of the previous tree'''
  nodeToPrune=None
  if not node.children: # return if tree is just single node
    return node
  # currMax = accuracy
  currMax = acc
  stack = [] # stack for DFS.
  stack.append(node)
  while len(stack) != 0: # At this point, we're just iterating through nodes to see pruning which one gives us the greatest gain in accuracy
    currnode = stack.pop()
    # currAcc = test(removeChildren(node, currnode.currAttribute), examples) # the other inefficient method I was trying
    temp = currnode.children # temporarily storing children
    if not bool(temp): # if no children
      continue
    currnode.children = {} # pruning it to see what accuracy it gives us
    currAcc = test(node, examples)
    if currAcc > currMax:
      currMax = currAcc
      nodeToPrune = currnode
    currnode.children = temp # un-pruning it to test other nodes
    for key in currnode.children:
      stack.append(currnode.children[key])
  if nodeToPrune==None: # this means that pruning any node didn't improve the performance of the tree
      return node
  nodeToPrune.children = {} # otherwise it did
  # print "\n","pruned",'\n'
  # accuracy = test(node, examples)
  # if acc > currMax: # might be redundant case
  #   print "accuracy base case executed"
  #   return node
  return prunehelp(node, examples, currMax) # so you keep pruning

def prune(node, examples):
  '''
  Takes in a trained tree and a validation set of examples.  Prunes nodes in order
  to improve accuracy on the validation data; the precise pruning strategy is up to you.
  '''
  return prunehelp(node, examples, test(node, examples))


def test(node, examples):
  '''
  Takes in a trained tree and a test set of examples.  Returns the accuracy (fraction
  of examples the tree classifies correctly).
  '''
  total = len(examples)
  correct = 0
  for example in examples:
    if evaluate(node, example) == example["Class"]:
      correct += 1
  return float(correct)/total

def evaluate(node, example):
  '''
  Takes in a tree and one example.  Returns the Class value that the tree
  assigns to the example.
  '''
  #attrDict=setAttrAndCount(example)
  # foundAttr = False
  while len(node.children) != 0:
    for value, child in node.children.iteritems(): # node.children is a dict
      #if example[node.currAttribute] not in attrDict[0]
      if example[node.currAttribute] == value:
        node = child
        # foundAttr = True
        break
    node = child
  return node.label


In [13]:
class Node:
    def __init__(self):
        self.label = None
        self.children = {}
	# you may want to add additional fields here...
        self.currAttribute = None
        # self.prevValue = None

In [31]:
data = [dict(a=1, b=0, c='?', Class=1), dict(a=1, b=3, c=2, Class=1),
         dict(a=2, b='?', c=1, Class=2), dict(a=2, b=1, c=3, Class=2),
         dict(a=3, b=0, c=1, Class=3), dict(a=3, b=2, c='?', Class=3)]

In [15]:
tree = ID3(data,0)

{'a': {1: 2, 2: 2, 3: 2}, 'c': {1: 2, 2: 1, 3: 1, '?': 2}, 'b': {0: 2, 1: 1, 2: 1, 3: 1, '?': 1}, 'Class': {1: 2, 2: 2, 3: 2}}
{'a': {1: 2, 2: 2, 3: 2}, 'c': {1: 2, 2: 1, 3: 1, '?': 2}, 'b': {0: 2, 1: 1, 2: 1, 3: 1, '?': 1}, 'Class': {1: 2, 2: 2, 3: 2}}
{'a': {1: 2}, 'c': {2: 1, '?': 1}, 'b': {0: 1, 3: 1}, 'Class': {1: 2}}
{'a': {2: 2}, 'c': {1: 1, 3: 1}, 'b': {1: 1, '?': 1}, 'Class': {2: 2}}
{'a': {3: 2}, 'c': {1: 1, '?': 1}, 'b': {0: 1, 2: 1}, 'Class': {3: 2}}
{'a': {1: 2, 2: 2, 3: 2}, 'c': {1: 2, 2: 1, 3: 1, '?': 2}, 'b': {0: 2, 1: 1, 2: 1, 3: 1, '?': 1}, 'Class': {1: 2, 2: 2, 3: 2}}
{'a': {1: 2, 2: 2, 3: 2}, 'c': {1: 2, 2: 1, 3: 1, '?': 2}, 'b': {0: 2, 1: 1, 2: 1, 3: 1, '?': 1}, 'Class': {1: 2, 2: 2, 3: 2}}
{'a': {1: 2}, 'c': {2: 1, '?': 1}, 'b': {0: 1, 3: 1}, 'Class': {1: 2}}
{'a': {2: 2}, 'c': {1: 1, 3: 1}, 'b': {1: 1, '?': 1}, 'Class': {2: 2}}
{'a': {3: 2}, 'c': {1: 1, '?': 1}, 'b': {0: 1, 2: 1}, 'Class': {3: 2}}
{'a': {1: 2, 2: 2, 3: 2}, 'c': {1: 2, 2: 1, 3: 1, '?': 2}, 'b': {0

In [5]:
evaluate(tree, dict(a=1, b=1, c=1))

1

In [7]:
evaluate(tree, dict(a=2, b=0, c=0))

2

In [45]:
import math

def setAttrAndCount(examples):
  '''Takes in an array of examples (which are dicts).
  Returns a nested dictionary of the format attr={value1:numOfOccurences, value2: numOfOccurences, ...}'''
  attrDict={}
  for example in examples:
    for attr, value in example.iteritems():
      if attrDict.has_key(attr): # if outerdict has attr as key
        if attrDict[attr].has_key(value): # if the innerdict already has the value as a key
          attrDict[attr][value] += 1
        else: # otherwise create it
          attrDict[attr][value] = 1
      else:
        attrDict[attr] = {value:1} # initialise the dictionary of values
  return attrDict


def entropy(examples, attribute):
  '''Takes an array of examples and calculates the entropy'''
  allinfo = setAttrAndCount(examples)
  print allinfo
  entropy = 0
  for value, count in  allinfo[attribute].iteritems():
    print value 
    print count 
    prob = float(count)/len(examples)
    print prob
    entropy -= prob*math.log(prob, 2)
    print entropy
  return entropy


def infoGain(examples, Attr):
  '''Calculates information gain of an Attribute from the given examples.'''
  allinfo = setAttrAndCount(examples)
  summation = 0
  for value, count in allinfo[Attr].iteritems():
    prob = float(count/len(examples))
    summation += prob*entropy([example for example in examples if example[Attr]==value], "Class")
  gain = entropy(examples, "Class") - summation
  return gain

def chooseAtrr(examples, attributes):
  '''Finds attribute with maximum info gain given a list of examples and attributes.'''
  maxim=0
  maxattr = None # attributes[0] # just a starting pt.
  for x in attributes:
    if infoGain(examples, x) > maxim:
      maxim = infoGain(examples, x)
      maxattr = x
  return maxattr


In [46]:
entropy(data,'a')
infoGain(data,'a')

{'a': {1: 2, 2: 2, 3: 2}, 'c': {1: 2, 2: 1, 3: 1, '?': 2}, 'b': {0: 2, 1: 1, 2: 1, 3: 1, '?': 1}, 'Class': {1: 2, 2: 2, 3: 2}}
1
2
0.333333333333
0.528320833574
2
2
0.333333333333
1.05664166715
3
2
0.333333333333
1.58496250072
{'a': {1: 2}, 'c': {2: 1, '?': 1}, 'b': {0: 1, 3: 1}, 'Class': {1: 2}}
1
2
1.0
0.0
{'a': {2: 2}, 'c': {1: 1, 3: 1}, 'b': {1: 1, '?': 1}, 'Class': {2: 2}}
2
2
1.0
0.0
{'a': {3: 2}, 'c': {1: 1, '?': 1}, 'b': {0: 1, 2: 1}, 'Class': {3: 2}}
3
2
1.0
0.0
{'a': {1: 2, 2: 2, 3: 2}, 'c': {1: 2, 2: 1, 3: 1, '?': 2}, 'b': {0: 2, 1: 1, 2: 1, 3: 1, '?': 1}, 'Class': {1: 2, 2: 2, 3: 2}}
1
2
0.333333333333
0.528320833574
2
2
0.333333333333
1.05664166715
3
2
0.333333333333
1.58496250072


1.584962500721156

In [48]:
entropy(data,'b')
infoGain(data,'b')

{'a': {1: 2, 2: 2, 3: 2}, 'c': {1: 2, 2: 1, 3: 1, '?': 2}, 'b': {0: 2, 1: 1, 2: 1, 3: 1, '?': 1}, 'Class': {1: 2, 2: 2, 3: 2}}
0
2
0.333333333333
0.528320833574
1
1
0.166666666667
0.959147917027
2
1
0.166666666667
1.38997500048
3
1
0.166666666667
1.82080208393
?
1
0.166666666667
2.25162916739
{'a': {1: 1, 3: 1}, 'c': {1: 1, '?': 1}, 'b': {0: 2}, 'Class': {1: 1, 3: 1}}
1
1
0.5
0.5
3
1
0.5
1.0
{'a': {2: 1}, 'c': {3: 1}, 'b': {1: 1}, 'Class': {2: 1}}
2
1
1.0
0.0
{'a': {3: 1}, 'c': {'?': 1}, 'b': {2: 1}, 'Class': {3: 1}}
3
1
1.0
0.0
{'a': {1: 1}, 'c': {2: 1}, 'b': {3: 1}, 'Class': {1: 1}}
1
1
1.0
0.0
{'a': {2: 1}, 'c': {1: 1}, 'b': {'?': 1}, 'Class': {2: 1}}
2
1
1.0
0.0
{'a': {1: 2, 2: 2, 3: 2}, 'c': {1: 2, 2: 1, 3: 1, '?': 2}, 'b': {0: 2, 1: 1, 2: 1, 3: 1, '?': 1}, 'Class': {1: 2, 2: 2, 3: 2}}
1
2
0.333333333333
0.528320833574
2
2
0.333333333333
1.05664166715
3
2
0.333333333333
1.58496250072


1.584962500721156

In [38]:
entropy(data,'c')
infoGain(data,'c')

{'a': {1: 2, 2: 2, 3: 2}, 'c': {1: 2, 2: 1, 3: 1, '?': 2}, 'b': {0: 2, 1: 1, 2: 1, 3: 1, '?': 1}, 'Class': {1: 2, 2: 2, 3: 2}}
{'a': {1: 2, 2: 2, 3: 2}, 'c': {1: 2, 2: 1, 3: 1, '?': 2}, 'b': {0: 2, 1: 1, 2: 1, 3: 1, '?': 1}, 'Class': {1: 2, 2: 2, 3: 2}}
{'a': {2: 1, 3: 1}, 'c': {1: 2}, 'b': {0: 1, '?': 1}, 'Class': {2: 1, 3: 1}}
{'a': {1: 1}, 'c': {2: 1}, 'b': {3: 1}, 'Class': {1: 1}}
{'a': {2: 1}, 'c': {3: 1}, 'b': {1: 1}, 'Class': {2: 1}}
{'a': {1: 1, 3: 1}, 'c': {'?': 2}, 'b': {0: 1, 2: 1}, 'Class': {1: 1, 3: 1}}
{'a': {1: 2, 2: 2, 3: 2}, 'c': {1: 2, 2: 1, 3: 1, '?': 2}, 'b': {0: 2, 1: 1, 2: 1, 3: 1, '?': 1}, 'Class': {1: 2, 2: 2, 3: 2}}


1.584962500721156

In [50]:
def getCountDict(examples, targetAtt):
  '''

  KEEP COUNT OF :
  total amount of each attribute - eg: handicapped - # of yes, # of no, # of q...
  within the particular value of an attribute - #democrat, # republican
  '''

  countDict = {}


  # TODO: TAKE OUT CLASS FROM ATTRIBUTES

  # Going through each example

  for example in examples:

    # Getting all the attributes from that example

    for (attribute, value) in example.iteritems():

      # Checking if that one attribute has already been looked at for other examples
        if attribute == targetAtt:
            pass

        elif attribute in countDict.keys():

          # if it has that attribute, check to see if that value has already been looked at for that attribute for other examples

            if value in countDict[attribute].keys():

              # If it has that value, check to see if that value of the targetAtt has been looked at for that value of that attribute for other examples

                if example[targetAtt] \
                    in countDict[attribute][value].keys():

                  # If that value of that target attribute has already been looked at for this value of this attribute of this example, just incrememt it

                    countDict[attribute][value][example[targetAtt]] += \
                        1
                else:

                  # Otherwise add that value of that target attribute to the dictionary

                    countDict[attribute][value][example[targetAtt]] = \
                        1
            else:

              # Does not have the value, add that value and value of the TargetAtt to the dictionary

                countDict[attribute][value] = {}
                countDict[attribute][value][example[targetAtt]] = 1
        else:
            countDict[attribute] = {}
            countDict[attribute][value] = {}
            countDict[attribute][value][example[targetAtt]] = 1

  return countDict

In [51]:
getCountDict(data,'Class')

{'a': {1: {1: 2}, 2: {2: 2}, 3: {3: 2}},
 'b': {0: {1: 1, 3: 1}, 1: {2: 1}, 2: {3: 1}, 3: {1: 1}, '?': {2: 1}},
 'c': {1: {2: 1, 3: 1}, 2: {1: 1}, 3: {2: 1}, '?': {1: 1, 3: 1}}}